In [ ]:
import tensorflow as tf

# Custom Sparse Categorical Crossentropy Loss Function
def custom_loss(y_true, y_pred):
    y_true = tf.one_hot(tf.squeeze(y_true), tf.shape(y_pred)[1]) # Convert y_true to one-hot encoding
    epsilon = tf.keras.backend.epsilon()     # Clip values to prevent NaN loss
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
    loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=1) # Calculate cross-entropy
    return tf.reduce_mean(loss)


# Custom Accuracy Metric Function
def custom_accuracy(y_true, y_pred):
    # Get predicted classes
    y_pred_classes = tf.argmax(y_pred, axis=1)

    # Flatten y_true if needed and cast to float32
    y_true = tf.cast(tf.squeeze(y_true), tf.float32)

    # Cast predicted classes to float32
    y_pred_classes = tf.cast(y_pred_classes, tf.float32)

    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred_classes), tf.float32))
    return accuracy

# Define your model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(100,), activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),  
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(len(categories), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model using custom loss and metric
model.compile(optimizer=optimizer, loss=custom_loss, metrics=[custom_accuracy])


history=model.fit(X_train, y_train, epochs=80, batch_size=32, validation_data=(X_test, y_test))